## Querying a Milvus index

Simple example on how to query content from a Milvus VectorStore.

Requirements:
- A Milvus instance, either standalone or cluster.
- Connection credentials to Milvus must be available as environment variables: MILVUS_USERNAME and MILVUS_PASSWORD

### Needed packages and imports

In [8]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

### Base parameters, the Milvus connection info

In [10]:
MILVUS_HOST = "vectordb-milvus.milvus.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = "root"
MILVUS_PASSWORD = "Milvus"
MILVUS_COLLECTION = "collection_nomicai_embeddings"

### Initialize the connection

In [11]:
# If you don't want to use a GPU, you can remove the 'device': 'cuda' argument
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    show_progress=True
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

### Make a query to the index to verify sources

In [12]:
query="How can I create a Data Science Project?"
results = store.similarity_search_with_score(query, k=4, return_metadata=True)
for result in results:
    print(result[0].metadata['source'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/html-single/managing_resources/index
https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/html-single/managing_resources/index
https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/html-single/managing_resources/index
https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/html-single/openshift_ai_tutorial_-_fraud_detection_example/index


### Work with a retriever

In [13]:
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [14]:
docs = retriever.get_relevant_documents(query)
docs

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='\\"kueue-webhook-service\\""\n}]},\n"code"\n:500}\n{\n"level"\n:\n"info"\n,\n"ts"\n:\n"2024-06-24T14:36:24.255137871Z"\n,\n"logger"\n:\n"controller-\nruntime.webhook"\n,\n"caller"\n:\n"webhook/server.go:242"\n,\n"msg"\n:\n"Serving webhook\n \nserver"\n,\n"host"\n:\n""\n,\n"port"\n:9443}\nCHAPTER 3. MANAGING CLUSTER RESOURCES\n25', metadata={'source': 'https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.13/html-single/managing_resources/index', 'page': 28}),
 Document(page_content='CHAPTER 3. MANAGING CLUSTER RESOURCES\n3.1. CONFIGURING THE DEFAULT PVC SIZE FOR YOUR CLUSTER\nTo configure how resources are claimed within your OpenShift AI cluster, you can change the default\nsize of the cluster’s persistent volume claim (PVC) ensuring that the storage requested matches your\ncommon storage workflow. PVCs are requests for resources in your cluster and also act as claim checks\nto the resource.\nPrerequisites\nYou have logged in to Red 